In [ ]:
import networkx as nx
import gurobipy as gp
from gurobipy import GRB
import multiprocessing as mp
import os
import math

In [ ]:
lst = []
for i in range(0,500):
    lst.append(0)
for i in range(500,1000):
    lst.append(1)
    
def test(index):
    return lst[index]

In [ ]:
def main():

    t_max = 1000
    t_min = 0
    del_t = t_max - t_min

    #p = 4
    ncpus = int(os.environ.get('SLURM_CPUS_PER_TASK',default=1))
    pool = mp.Pool(processes=ncpus)

    flag = 0
    while(del_t > ncpus):
        time_p = []
        for i in range(0,ncpus):
            time_p.append(((i+1) * (math.ceil(((t_max - t_min)/(ncpus+1))))) + t_min)

        print(time_p)

        results = [pool.apply_async(test, args=(x)) for x in time_p]
        f_vals = [p.get() for p in results]

        length = len(f_vals)
        flag = 0
        for i in range(0, length-1):
            if f_vals[i] == 0 and f_vals[i+1] == 1:
                t_min = time_p[i]
                t_max = time_p[i+1]
                flag = 1
                break

            elif f_vals[i] == 1 and f_vals[i+1] == 1:
                #t_min = 0           #lowerbound
                t_max = time_p[i]
               # upper_bound = t_max
                flag = 1
                break


        if flag == 0:
            t_min = time_p[ncpus-1]
            break

        del_t = t_max - t_min


    #upper_bound = t_max
    ncpus = del_t
    pool = mp.Pool(processes=ncpus)
    time_p = []
    for i in range(0,ncpus):
        time_p.append(((i+1) * (math.ceil(((t_max - t_min)/(ncpus+1))))) + t_min)

    print(time_p)

    results = [pool.apply_async(test, args=(x)) for x in time_p]
    f_vals = [p.get() for p in results]

    length = len(f_vals)
    #flag = 0
    for i in range(0, length-1):
        if f_vals[i] == 1:
            t_max = time_p[i]
            #flag = 1
            break

    #if(flag == 0):
        #t_max = upper_bound

    print("1 found at position - ", t_max)
    return 0  

main()